# Introduction to Agents

This tutorial is a simple application of a LangChain multi-agent approach. Aim of this tutorial is to show how tools and LLMs can be combined into a multi-agent framework that includes an supervisor agent and several sub-agents. The supervisor gets a request from the user, then splits this information to the respective sub-agents that can search for specific information. Then the supervisor collects all the information and synthesizes an answer to the user.

### Use case

The multi-agent system that is developed is a "tourist guide", who knows the user's geographic location (we assume that the user has granted access to her/his location) and proposes restaurants, museums and sports activities. The user can request information for these at a specific day / time. The weather is also taken into account for the proposals.

### Tools, agents and LLMs

We consider four sub-agents, each of which utilize one specialized tool:
- The `restaurant_agent` uses the `search_restaurants` tool.
- The `museum_agent` uses the `search_museums` tool.
- The  `sports_agent` uses the `search_sports` tool.
- The `weather_agent` uses the `get_weather` tool.

After the agents have been created, the `supervisor_agent` is created, which needs to have access to those sub-agents as tools. Therefore, we envelope the sub-agents in their repective tools.

To reserve resources and time, a single LLM is initialized and plays all the roles, i.e., a single LLM is instantiated that runs with the prompts that are given by the user, are particular for each agent, or are generated by the supervisor agent for running the sub-agents.

### Demo conditions

All the search tools (except the weather) use the [Google SerpApi](https://serpapi.com/), the free version of which has a daily limit of searches that can be performed. The tutorial is set up in such a way that it is not necessary to run searches. Code is provided in comments that show how to use the SerpApi inside the tools. The tools in the tutorial, however, load pickle files with "frozen" results of each tool, taken for the specific query of the example.

In [1]:
# import requests
# from dotenv import load_dotenv
# import os
# import pickle

# load_dotenv()

# API_KEY = os.getenv("SEPAPI_API_KEY")

# params = {
#     "engine": "google_maps",
#     "q": "Traditional Greek Restaurant",
#     "ll": "@35.363366,24.476021, 14z",  # Somewhere in Rethymno, Greece
#     "type": "search",
#     "api_key": API_KEY
# }

# response = requests.get("https://serpapi.com/search", params=params)
# response.raise_for_status()

# restaurants = response.json()

# for place in restaurants.get("local_results", [])[:5]:
#     print(place.get("title"))
#     print("Address:", place.get("address"))
#     print("Rating:", place.get("rating"))
#     print("Coordinates:", place.get("gps_coordinates"))
#     print("-" * 40)

# with open('data/restaurants.pickle', 'wb') as handle:
#     pickle.dump(restaurants, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [2]:
# params = {
#     "engine": "google_maps",
#     "q": "Museum or Archaeological Site",
#     "ll": "@35.363366,24.476021, 14z",  # Somewhere in Rethymno, Greece
#     "type": "search",
#     "api_key": API_KEY
# }

# response = requests.get("https://serpapi.com/search", params=params)
# response.raise_for_status()

# museums = response.json()

# for place in museums.get("local_results", [])[:5]:
#     print(place.get("title"))
#     print("Address:", place.get("address"))
#     print("Rating:", place.get("rating"))
#     print("Coordinates:", place.get("gps_coordinates"))
#     print("-" * 40)

# with open('data/museums.pickle', 'wb') as handle:
#     pickle.dump(museums, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [3]:
# params = {
#     "engine": "google_maps",
#     "q": "Adventure or sports activity",
#     "ll": "@35.363366,24.476021, 14z",  # Somewhere in Rethymno, Greece
#     "type": "search",
#     "api_key": API_KEY
# }

# response = requests.get("https://serpapi.com/search", params=params)
# response.raise_for_status()

# sports = response.json()

# for place in sports.get("local_results", [])[:5]:
#     print(place.get("title"))
#     print("Address:", place.get("address"))
#     print("Rating:", place.get("rating"))
#     print("Coordinates:", place.get("gps_coordinates"))
#     print("-" * 40)

# with open('data/sports.pickle', 'wb') as handle:
#     pickle.dump(sports, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Loading results

As said earlier, results obtained for the specific query are retrieved from pickle files, to make sure that SerpApi is not required for the tutorial.

In [4]:
import pickle

with open('data/restaurants.pickle', 'rb') as handle:
    restaurants = pickle.load(handle)
with open('data/museums.pickle', 'rb') as handle:
    museums = pickle.load(handle)
with open('data/sports.pickle', 'rb') as handle:
    sports = pickle.load(handle)

### Distance calculator

A good idea is to include distance in the tools, so that there is an estimate about how far each proposed location is based on the current location of the user. The `haversine` distance is used for simplicity, but other specialized python libriaries are available for getting directions, specific distances and times of arrival for different means of commuting.

In [5]:
from math import radians, sin, cos, sqrt, atan2

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # km
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    return R * c

### User location

We assume that the user has voluntarily shared their current location in latitude and longtitude coordinates.

In [6]:
# user location
user_lat = 35.363366
user_lon = 24.476021

### Examine SerpApi results

We can examine the result received on a low-level from SerpApi, to have an idea about what data our LLM agent will have access to and what answer the agent can form. We can also add the distance calculator here.

In [8]:
for place in restaurants.get("local_results", [])[:5]:
    print(place.get("title"))
    print("Address:", place.get("address"))
    print("Rating:", place.get("rating"))
    print("Coordinates:", place.get("gps_coordinates"))
    print("Distance (km):", haversine(
        user_lat, user_lon,
        place.get("gps_coordinates", {}).get("latitude"),
        place.get("gps_coordinates", {}).get("longitude")
    ))
    print("Price:", place.get("price"))
    print("Open/closed:", place.get("open_state"))
    print("Operating hours:", place.get("operating_hours"))
    print("service_options:", place.get("service_options"))
    print("-" * 40)

Othonas taverna
Address: Mavrokordatou Alexanrou 27, Rethymno 741 31, Greece
Rating: 4.8
Coordinates: {'latitude': 35.369527, 'longitude': 24.4746158}
Distance (km): 0.6968205034160434
Price: €10–20
Open/closed: Open · Closes 12:55 AM
Operating hours: {'monday': '11\u202fAM–12:55\u202fAM', 'tuesday': '11\u202fAM–12:55\u202fAM', 'wednesday': '11\u202fAM–12:55\u202fAM', 'thursday': '11\u202fAM–12:55\u202fAM', 'friday': '11\u202fAM–12:55\u202fAM', 'saturday': '11\u202fAM–12:55\u202fAM', 'sunday': '11\u202fAM–11:30\u202fPM'}
service_options: {'dine_in': True, 'curbside_pickup': True, 'delivery': False}
----------------------------------------
Taverna Yiannis!!! Greek traditional food, fresh fish, fresh meat and pasta!!!
Address: Leof. Machis Kritis 159, Rethymno 741 00, Greece
Rating: 4.8
Coordinates: {'latitude': 35.3684806, 'longitude': 24.531903699999997}
Distance (km): 5.099055164092606
Price: €10–20
Open/closed: Open · Closes 11 PM
Operating hours: {'monday': '11\u202fAM–11\u202fPM', 

In [9]:
for place in museums.get("local_results", [])[:5]:
    print(place.get("title"))
    print("Address:", place.get("address"))
    print("Rating:", place.get("rating"))
    print("Coordinates:", place.get("gps_coordinates"))
    print("Distance (km):", haversine(
        user_lat, user_lon,
        place.get("gps_coordinates", {}).get("latitude"),
        place.get("gps_coordinates", {}).get("longitude")
    ))
    print("Price:", place.get("price"))
    print("Open/closed:", place.get("open_state"))
    print("Operating hours:", place.get("operating_hours"))
    print("service_options:", place.get("service_options"))
    print("-" * 40)

Archaeological Museum of Rethymnon
Address: Αγ. Φραγκίσκου, Rethymno 741 31, Greece
Rating: 4.4
Coordinates: {'latitude': 35.3684268, 'longitude': 24.474361299999998}
Distance (km): 0.5825117443756878
Price: None
Open/closed: Closed · Opens 9 AM Wed
Operating hours: {'monday': '9\u202fAM–5\u202fPM', 'tuesday': 'Closed', 'wednesday': '9\u202fAM–5\u202fPM', 'thursday': '9\u202fAM–5\u202fPM', 'friday': '9\u202fAM–5\u202fPM', 'saturday': '9\u202fAM–5\u202fPM', 'sunday': '9\u202fAM–5\u202fPM'}
service_options: None
----------------------------------------
Eleutherna Archaeological Museum
Address: Epar.Od. Aggelianon-Elefthernas 123, Eleftherna 740 52, Greece
Rating: 4.6
Coordinates: {'latitude': 35.3240948, 'longitude': 24.6705675}
Distance (km): 18.17790712853423
Price: None
Open/closed: Closed · Opens 8:30 AM Wed
Operating hours: {'monday': '8:30\u202fAM–3:30\u202fPM', 'tuesday': 'Closed', 'wednesday': '8:30\u202fAM–3:30\u202fPM', 'thursday': '8:30\u202fAM–3:30\u202fPM', 'friday': '8:30\u

In [10]:
for place in sports.get("local_results", [])[:5]:
    print(place.get("title"))
    print("Address:", place.get("address"))
    print("Rating:", place.get("rating"))
    print("Coordinates:", place.get("gps_coordinates"))
    print("Distance (km):", haversine(
        user_lat, user_lon,
        place.get("gps_coordinates", {}).get("latitude"),
        place.get("gps_coordinates", {}).get("longitude")
    ))
    print("Price:", place.get("price"))
    print("Open/closed:", place.get("open_state"))
    print("Operating hours:", place.get("operating_hours"))
    print("service_options:", place.get("service_options"))
    print("-" * 40)

Escape in Action Canyoning
Address: Ανθυπολοχαγού Ψυχουντάκη, Rethymno 741 32, Greece
Rating: 4.9
Coordinates: {'latitude': 35.365245, 'longitude': 24.4679559}
Distance (km): 0.7605889339645273
Price: None
Open/closed: Open 24 hours
Operating hours: {'monday': 'Open 24 hours', 'tuesday': 'Open 24 hours', 'wednesday': 'Open 24 hours', 'thursday': 'Open 24 hours', 'friday': 'Open 24 hours', 'saturday': 'Open 24 hours', 'sunday': 'Open 24 hours'}
service_options: None
----------------------------------------
B.O.A - Base Outdoor Activities Creta
Address: Kriari 11, Rethymno 741 32, Greece
Rating: 5
Coordinates: {'latitude': 35.3651791, 'longitude': 24.4703849}
Distance (km): 0.5493994050711971
Price: None
Open/closed: Closed · Opens 8 AM Tue
Operating hours: {'monday': '8\u202fAM–6\u202fPM', 'tuesday': '8\u202fAM–6\u202fPM', 'wednesday': '8\u202fAM–6\u202fPM', 'thursday': '8\u202fAM–6\u202fPM', 'friday': '8\u202fAM–6\u202fPM', 'saturday': '8\u202fAM–6\u202fPM', 'sunday': '8\u202fAM–6\u202

### Weather tool results

Similary with weather, we can examine the results and decide level of detail.

In [11]:
import requests

params = {
    'latitude': user_lat,
    'longitude': user_lon,
    "hourly": [
        "temperature_2m",
        "windspeed_10m",
        "precipitation",
        "precipitation_probability"
    ],
    "daily": [
        "precipitation_sum",
        "precipitation_probability_max",
        "temperature_2m_max",
        "temperature_2m_min"
    ],
    "forecast_hours": 3,   # next 3 hours only
}

response = requests.get("https://api.open-meteo.com/v1/forecast", params=params)
response.raise_for_status()

weather = response.json()

In [ ]:
print(weather["hourly"])
print(weather["daily"])

{'time': ['2026-02-03T15:00', '2026-02-03T16:00', '2026-02-03T17:00'], 'temperature_2m': [15.6, 15.0, 14.2], 'windspeed_10m': [3.3, 2.5, 3.9], 'precipitation': [0.0, 0.0, 0.0], 'precipitation_probability': [0, 0, 0]}


### Initialize LLM

We initialize a single LLM that will be employed for the role of all agents.

In [14]:
from langchain_ollama import ChatOllama
from langchain.tools import tool

# Initialize the ChatOllama model with the specified model name
model_name = 'qwen2.5:3b'

# and initialize the ChatOllama instance
chat_model = ChatOllama(
    model=model_name,
    validate_model_on_init=True,
    temperature=0.3,
    disable_streaming=True,
)

### Initialize agent tools

First we initialize the tools that each agent will use for their specialized searches.

In [15]:
@tool
def search_restaurants(query: str) -> str:
    """Search for restaurants on the internet using a search tool.
    The type of restaurant may or may not be specified in the query.
    The results include name, address, rating, and other details.
    It also includes distance from the user's location."""
    with open('data/restaurants.pickle', 'rb') as handle:
        restaurants = pickle.load(handle)
    print("Executing restaurant search tool with query:", query)
    answer = 'Results for restaurant search:\n'
    for place in restaurants.get("local_results", [])[:5]:
        answer += f"{place.get('title')}\n, \
            Address: {place.get('address')}\n, \
            Rating: {place.get('rating')}\n, \
            Distance (km): {haversine(user_lat, user_lon, \
                            place.get('gps_coordinates', {}).get('latitude'), \
                            place.get('gps_coordinates', {}).get('longitude'))}\n"
        answer += f"Price: {place.get('price')}\n, \
            Open/closed: {place.get('open_state')}\n, \
            Operating hours: {place.get('operating_hours')}\n, \
            service_options: {place.get('service_options')}\n"
        answer += "-" * 40 + "\n"
    return answer

In [16]:
@tool
def search_museums(query: str) -> str:
    """Search for museums on the internet using a search tool.
    The results include name, address, rating, and other details.
    It also includes distance from the user's location."""
    with open('data/museums.pickle', 'rb') as handle:
        museums = pickle.load(handle)
    print("Executing museum search tool with query:", query)
    answer = 'Results for museum search:\n'
    for place in museums.get("local_results", [])[:5]:
        answer += f"{place.get('title')}\n, \
            Address: {place.get('address')}\n, \
            Rating: {place.get('rating')}\n, \
            Distance (km): {haversine(user_lat, user_lon, \
                            place.get('gps_coordinates', {}).get('latitude'), \
                            place.get('gps_coordinates', {}).get('longitude'))}\n"
        answer += f"Price: {place.get('price')}\n, \
            Open/closed: {place.get('open_state')}\n, \
            Operating hours: {place.get('operating_hours')}\n, \
            service_options: {place.get('service_options')}\n"
        answer += "-" * 40 + "\n"
    return answer

In [17]:
@tool
def search_sports(query: str) -> str:
    """Search for sports activities on the internet using a search tool.
    The results include name, address, rating, and other details.
    It also includes distance from the user's location."""
    with open('data/sports.pickle', 'rb') as handle:
        sports = pickle.load(handle)
    print("Executing sports search tool with query:", query)
    answer = 'Results for sports search:\n'
    for place in sports.get("local_results", [])[:5]:
        answer += f"{place.get('title')}\n, \
            Address: {place.get('address')}\n, \
            Rating: {place.get('rating')}\n, \
            Distance (km): {haversine(user_lat, user_lon, \
                            place.get('gps_coordinates', {}).get('latitude'), \
                            place.get('gps_coordinates', {}).get('longitude'))}\n"
        answer += f"Price: {place.get('price')}\n, \
            Open/closed: {place.get('open_state')}\n, \
            Operating hours: {place.get('operating_hours')}\n, \
            service_options: {place.get('service_options')}\n"
        answer += "-" * 40 + "\n"
    return answer

In [19]:
@tool
def get_weather() -> str:
    """Get the current weather information for the user's location."""
    params = {
        'latitude': user_lat,
        'longitude': user_lon,
        "hourly": [
            "temperature_2m",
            "windspeed_10m",
            "precipitation",
            "precipitation_probability"
        ],
        "daily": [
            "precipitation_sum",
            "precipitation_probability_max",
            "temperature_2m_max",
            "temperature_2m_min"
        ],
        "forecast_hours": 3,   # next 3 hours only
    }

    response = requests.get("https://api.open-meteo.com/v1/forecast", params=params)
    response.raise_for_status()

    weather = response.json()
    return weather

### Create agents

We can no create agents with the tools we have just created. For each agent we can use a specialized system prompt for providing them with a better direction regarding the style, level of details and overall appearance of the results they provide. In this trivial example, the SerpApi-related agents do a very similary job, so their system prompts are very much alike.

System prompts allow us to employ a single LLM for every agent. LLMs are stateless, i.e., they remember no history of interaction with the world unless we explicitly provide it to them through context/prompts. System prompts do exactly that: they remind the LLM what role they need to play every time they are called.

In [20]:
from langchain.agents import create_agent

In [ ]:
restaurant_agent = create_agent(
    model=chat_model,
    tools=[
        search_restaurants
    ],
    system_prompt="""
    You are a helpful assistant that helps users find open
    restaurants based only on the results provided by a
    restaurant search tool to get. It is not necessary to have
    a description of the restaurant or the user location, the
    tool will provide all necessary information.
    """
)

museum_agent = create_agent(
    model=chat_model,
    tools=[
        search_museums
    ],
    system_prompt="""
    You are a helpful assistant that helps users find open
    museums based only on the results provided by a
    museum search tool to get. It is not necessary to have
    a description of the museum or the user location, the
    tool will provide all necessary information.
    """
)

sports_agent = create_agent(
    model=chat_model,
    tools=[
        search_sports
    ],
    system_prompt="""
    You are a helpful assistant that helps users find open
    sports activities based only on the results provided by a
    sports search tool to get. It is not necessary to have
    a description of the sports venue or the user location, the
    tool will provide all necessary information.
    """
)

weather_agent = create_agent(
    model=chat_model,
    tools=[
        get_weather
    ],
    system_prompt="""
    You are a helpful assistant that helps users find open
    weather information based only on the results provided by a
    weather search tool that provides information for the next few hours
    today and for the next few days. It is not necessary to have
    the user location, the tool will provide all necessary information.
    """
)


### Trying out an agent

We can invoke each agent independently to examine the answers they provide. This will help us understand what the role of the supervisor agent will be.

In [22]:
result = restaurant_agent.invoke({
    "messages": [
        {
            "role": "system",
            "content": "Find some good restaurants based on the user's request below."
        },
        {
            "role": "user",
            "content": "Can you suggest some traditional restaurants in Rethymno, Crete?"
        }
    ]
})

Executing restaurant search tool with query: traditional restaurants Rethymno Crete


In [ ]:
print(result['messages'][-1].content)

Results for restaurant search:
Othonas taverna
,             Address: Mavrokordatou Alexanrou 27, Rethymno 741 31, Greece
,             Rating: 4.8
,             Distance (km): 0.6968205034160434
Price: €10–20
,             Open/closed: Open · Closes 12:55 AM
,             Operating hours: {'monday': '11\u202fAM–12:55\u202fAM', 'tuesday': '11\u202fAM–12:55\u202fAM', 'wednesday': '11\u202fAM–12:55\u202fAM', 'thursday': '11\u202fAM–12:55\u202fAM', 'friday': '11\u202fAM–12:55\u202fAM', 'saturday': '11\u202fAM–12:55\u202fAM', 'sunday': '11\u202fAM–11:30\u202fPM'}
,             service_options: {'dine_in': True, 'curbside_pickup': True, 'delivery': False}
----------------------------------------
Taverna Yiannis!!! Greek traditional food, fresh fish, fresh meat and pasta!!!
,             Address: Leof. Machis Kritis 159, Rethymno 741 00, Greece
,             Rating: 4.8
,             Distance (km): 5.099055164092606
Price: €10–20
,             Open/closed: Open · Closes 11 PM
,             O

In [36]:
result = weather_agent.invoke({
        "messages": [
            {
                "role": "system",
                "content": "Find current weather information for the user's location."
            },
            {
                "role": "user",
                "content": "What will be the weather like in three days?"
            }
        ]
    })

In [37]:
print(result['messages'][-1].text)

Based on the weather data for three days ahead:

- On February 3rd (today), the temperature will be around 15.6°C with no precipitation.
- On February 4th, the temperature is expected to drop slightly to around 15.0°C and there's a chance of light rain.
- On February 5th, it looks like there might be some rainfall as the precipitation sum is expected to reach up to 2.7 mm.

For the rest of the days in three days ahead:
- February 6th: Temperature will range between 13.9°C and 18.7°C with no significant changes.
- February 7th: The temperature could drop slightly, reaching down to around 17.0°C.
- February 8th: Temperatures are expected to be warmer again, ranging from 15.1°C to 21.3°C.
- February 9th: Similar to the previous day, temperatures will likely range between 13.6°C and 17.0°C.

Please note that these predictions might change due to various weather factors.


### Agents as tools

The supervisor agent needs to have access to all other agents as tools. The supervisor agent is indeed an agent, so the way it works is by calling tools, which, in this case, happen to include calls to other agents (the sub-agents).

As mentioned in tutorial 02, it is important to include accurate and complete descriptions of what each tool-agent does in the comments of each tool function. These comments will inform the supervisor agent about what each tool does and how they can be used, depending on the user query.

Additionally, in each tool function we have a chance to provide additional context to each sub-agent by providing extra system prompts along with the supervisor agent request.

In [ ]:
@tool
def restaurant_agent_tool(user_request: str) -> str:
    """ Find restaurants based on user request.
    
    Use the restaurant_agent_tool to get restaurant suggestions.
    Input: Natural language string with user request (e.g., traditional restaurants).
    """
    restaurants = restaurant_agent.invoke({
        "messages": [
            {
                "role": "system",
                "content": """
                Find some good restaurants based on the user's request below.
                If the user specifies time, make sure the restaurant is open at that time.
                If the user specifies dietary preferences, make sure the restaurant meets those preferences.
                """
            },
            {
                "role": "user",
                "content": user_request
            }
        ]
    })
    return restaurants['messages'][-1].text

@tool
def museum_agent_tool(user_request: str) -> str:
    """ Find museums based on user request.
    
    Use the museum_agent_tool to get museum suggestions.
    Input: Natural language string with user request (e.g., museums of contemporaty art).
    """
    museums = museum_agent.invoke({
        "messages": [
            {
                "role": "system",
                "content": """
                Find some good museums based on the user's request below.
                If the user specifies time, make sure the museum is open at that time.
                If the user specifies dietary preferences, make sure the museum meets those preferences.
                """
            },
            {
                "role": "user",
                "content": user_request
            }
        ]
    })
    return museums['messages'][-1].text

@tool
def sports_agent_tool(user_request: str) -> str:
    """ Find sports activities based on user request.
    
    Use the sports_agent_tool to get sports activity suggestions.
    Input: Natural language string with user request (e.g., outdoor sports in summer).
    """
    sports = sports_agent.invoke({
        "messages": [
            {
                "role": "system",
                "content": """
                Find some good sports activities based on the user's request below.
                If the user specifies time, make sure the activity is available at that time.
                If the user specifies dietary preferences, make sure the activity meets those preferences.
                """
            },
            {
                "role": "user",
                "content": user_request
            }
        ]
    })
    return sports['messages'][-1].text

@tool
def weather_agent_tool(user_request: str) -> str:
    """ Find weather information based on user request.
    
    Use the weather_agent_tool to get weather information.
    Input: No inpit is given but user location is known.
    """
    weather = weather_agent.invoke({
        "messages": [
            {
                "role": "system",
                "content": "Find current weather information for the user's location."
            },
            {
                "role": "user",
                "content": user_request
            }
        ]
    })
    return weather['messages'][-1].text

### Supervisor agent

We can initialize the supervisor agent with a basic prompt.

In [50]:
supervisor_agent = create_agent(
    model=chat_model,
    tools=[
        restaurant_agent_tool,
        museum_agent_tool,
        sports_agent_tool,
        weather_agent_tool
    ],
    system_prompt="""
    You are a helpful assistant that helps users find open
    restaurants, museums, sports activities, and weather information
    based only on the results provided by their respective search tools.
    """
)

### Run the multi-agent system

We can run the system by triggering the supervisor agent with a user query. In LangChain we can use `stream` to trigger the basic interaction with the agent. If we want to go deeper in how the agents interact we will need to use LangGraph and if we want to have access to better inspection tools about what each agent does and when, we need to use LangSmith. These will be the subject of another tutorial.

In [48]:
query = """
I would like to find a traditional Greek restaurant that is open for dinner tonight.
Also, can you suggest a museum to visit tomorrow afternoon and tell me what the 
weather will be like over the next few days?
"""

for step in supervisor_agent.stream(
    {"messages": [{"role": "user", "content": query}]}
):
    for update in step.values():
        for message in update.get("messages", []):
            message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  restaurant_agent_tool (f2c9552a-5bae-42fc-9cdc-c94139101427)
 Call ID: f2c9552a-5bae-42fc-9cdc-c94139101427
  Args:
    user_request: traditional Greek restaurant open for dinner tonight
  museum_agent_tool (0d4e7a02-45a4-4fb7-9de4-6907c7a303e1)
 Call ID: 0d4e7a02-45a4-4fb7-9de4-6907c7a303e1
  Args:
    user_request: museum to visit tomorrow afternoon
  weather_agent_tool (90c474a4-9290-4fd2-a000-641903997fd7)
 Call ID: 90c474a4-9290-4fd2-a000-641903997fd7
  Args:
    user_request: next few days
TOOL: restaurant_agent_tool called with request: traditional Greek restaurant open for dinner tonight
Executing museum search tool with query: museum to visit tomorrow afternoon
Executing restaurant search tool with query: traditional Greek restaurant dinner
================================= Tool Message =================================
Name: museum_agent_tool

Based on your request for a museum to v